In [9]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from src.Value import Value, draw_dot
from src.activation import * 
from src.init import he_init
from src.Layer import Layer
from src.FFNN import FFNN
from src.loss import cce_loss  
from sklearn.metrics import accuracy_score
def one_hot_encode(y, num_classes=10):
    """
    Konversi array label ke one-hot encoding.
    y: array dengan shape (n_samples, 1) atau (n_samples,)
    """
    y = y.flatten().astype(int)
    one_hot = np.zeros((y.shape[0], num_classes))
    one_hot[np.arange(y.shape[0]), y] = 1
    return one_hot

# Muat dataset MNIST dari OpenML
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Normalisasi fitur ke rentang [0,1]
X = X.astype(np.float32) / 255.0

# Konversi label ke integer
y = y.astype(np.int32)

# Ubah label ke one-hot encoding untuk 10 kelas
y_onehot = one_hot_encode(y, num_classes=10)

# Bagi data menjadi training dan validation (misal 80% training, 20% validasi)
X_train, X_val, y_train, y_val = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Bungkus data menggunakan class Value
X_train = Value(X_train)
y_train = Value(y_train)
X_val = Value(X_val)
y_val = Value(y_val)

# Definisikan arsitektur model.
# Karena MNIST memiliki 784 fitur, layer pertama menggunakan 784 neuron.
# Output layer memiliki 10 neuron dengan fungsi aktivasi softmax.
layers = [
    Layer(784, 128, activation=relu, weight_init=he_init),
    Layer(128, 32, activation= relu, weight_init=he_init),
    Layer(32, 10, activation=softmax, weight_init=he_init)
]

# Buat instance model FFNN dengan loss function cce_loss
model = FFNN(layers=layers, loss_fn=cce_loss, lr=0.01)

# Latih model dengan parameter batch_size, max_epoch, dan error_threshold yang diinginkan
training_history = model.train(
    training_data=X_train,
    training_target=y_train,
    max_epoch=20,            # jumlah epoch maksimum
    error_threshold=0.01,    # ambang error untuk penghentian training
    batch_size=64,           # ukuran mini-batch
    validation_data=X_val,   # data validasi (opsional)
    validation_target=y_val, # target validasi (opsional)
    verbose=True             # tampilkan progress training
)

print("Training Loss History:", training_history['training_loss_history'])
print("Validation Loss History:", training_history['validation_loss_history'])


Epoch 1/20: Training Loss = 0.8987747508968625, Validation Loss = 0.4266734977703012
Epoch 2/20: Training Loss = 0.3740118362903595, Validation Loss = 0.33387718526085475
Epoch 3/20: Training Loss = 0.31196106673989976, Validation Loss = 0.2967215373496829
Epoch 4/20: Training Loss = 0.27868279199940815, Validation Loss = 0.27325744981736777
Epoch 5/20: Training Loss = 0.2550456403706755, Validation Loss = 0.2554862572906412
Epoch 6/20: Training Loss = 0.2362235957639558, Validation Loss = 0.24058312241728325
Epoch 7/20: Training Loss = 0.22002168367164476, Validation Loss = 0.22747242703509446
Epoch 8/20: Training Loss = 0.2055683751659734, Validation Loss = 0.21551998712120382
Epoch 9/20: Training Loss = 0.19255307136688912, Validation Loss = 0.20500062173841316
Epoch 10/20: Training Loss = 0.180816958039999, Validation Loss = 0.19555567375986585
Epoch 11/20: Training Loss = 0.17021276038459368, Validation Loss = 0.1871069629045639
Epoch 12/20: Training Loss = 0.16059105796473366, Va

In [10]:
pred = model(X_val)
pred = np.argmax(pred.data, axis=1)

In [11]:
def decode(y_onehot):
    return np.argmax(y_onehot, axis=1)
y_val = decode(y_val.data)
accuracy = accuracy_score(y_val, pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 95.93%
